# Coarse-grained Preprocessing


## Preprocessing


In [1]:
from pandas import read_excel
import pandas as pd
import numpy as np
from datetime import datetime
import os
os.environ["KERAS_BACKEND"] = "tensorflow" 
import keras
def parse(x):
    return datetime.strptime(x,'%Y%m%d')
def get_dataframe(data_type,date_no):
    path=f'{data_type}/{date_no}'
    data=pd.read_excel(path)
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d')
    pd.set_option('future.no_silent_downcasting', True)
    data.replace('/', np.nan, inplace=True)
    data.set_index('Date', inplace=True)
    os.makedirs(f'data/new_data/{data_type}', exist_ok=True)
    data.describe().to_csv(f'data/new_data/{data_type}/{date_no}_statistic.csv',encoding='utf-8-sig')
    print(data.index)
    output_dir = f'data_set/{data_type}'
    os.makedirs(output_dir, exist_ok=True)
    data.to_csv(f'data_set/{data_type}/processed_{date_no}.csv')
    return data

## Feature Selection
NOTE:it's not simply PCA.
We plan to try 5 methods of supervised feature selection,namely:
1. Filter Method:
   
    a. Pearson's Coefficient
   
    b. Chi Squared
   
    c. ANOVA Coefficient
   
3. Wrapper Method:
   
    a. Recursive Feature Elimination*
   
    b. Genetic Algorithms

In [2]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
# input split
def my_rfe(df):
    df_d = pd.DataFrame()
    for feature in df.columns:
        kmeans = KMeans(n_clusters=8, random_state=0)
        df_d[feature + '_binned'] = kmeans.fit_predict(df[[feature]])
    X=df_d.drop(columns=['CGM (mg / dl)'+ '_binned'],axis=1)
    y=df_d['CGM (mg / dl)'+ '_binned'] 
    rfe=RFE(estimator=DecisionTreeClassifier(),n_features_to_select=4)
    rfe.fit(X,y)
    for i,col in zip(range(X.shape[1]),X.columns):
        print(f"{col} selected={rfe.support_[i]} rank={rfe.ranking_[i]}")

### Sample data Analysis(RFE)
```python
feature_selection_info = [
    {'feature': 'CBG (mg / dl)_binned', 'selected': True, 'rank': 1},
    {'feature': 'Blood Ketone (mmol / L)_binned', 'selected': False, 'rank': 4},
    {'feature': 'CSII - bolus insulin (Novolin R, IU)_binned', 'selected': True, 'rank': 1},
    {'feature': 'CSII - basal insulin (Novolin R, IU / H)_binned', 'selected': True, 'rank': 1},
    {'feature': 'IU- s.c._binned', 'selected': True, 'rank': 1},
    {'feature': 'IU- i.v._binned', 'selected': False, 'rank': 3},
    {'feature': 'Non-insulin hypoglycemic agents - mg_binned', 'selected': False, 'rank': 2},
    {'feature': 'Dietary calories_binned', 'selected': True, 'rank': 1},
]
```
From above ,we can omit the 3 features:
1. ```Blood Ketone (mmol / L)```,
2. ```IU- i.v.```,
3. ```Non-insulin hypoglycemic agents - mg``` 

## Fine-grained preprocessing
1. Use a coarse hashmap derived from the manual recorded csv file to calculate the total calories for a meal.
2. Extract number from ```Insulin dose - s.c.```,```Insulin dose - i.v.```,```Non-insulin hypoglycemic agents```
3. Padding the empty cell in ```CSII - basal insulin (Novolin R, IU / H)``` columns with the very above non-empty cell.
4. The ```CGM``` is the variables we want to predict,so remove other irrelavant columns(10, 11, 12, 13, 14, 15,16,17)
5. If the data is unavailable ,fill in it with the nearly average value of calories in a meal.
### NOTE:
1. The scaler's dimentionality must match the prediction.

In [3]:
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,MultiLabelBinarizer
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import re
import warnings
import csv
warnings.filterwarnings("ignore", message="All-NaN slice encountered")
#转成有监督数据
def series_to_supervised(data, n_in=1, n_out=1, n_lag=1,dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    #数据序列(也将就是input) input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + n_lag, i)) for j in range(n_vars)]
        #预测数据（input对应的输出值） forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + n_lag)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + n_lag, i)) for j in range(n_vars)]
    #拼接 put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # 删除值为NAN的行 drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
hash_table = {}
with open('ref_calories.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # 使用第一列作为键，第二列作为值，填充哈希表
        if len(row) >= 2:  # 确保行中至少有两列
            key = row[0]
            value = row[1]
            hash_table[key] = value
def extract_dietary_words(text):
    if pd.notna(text):
        res=0.0
        phrases = re.findall(r'\b[A-Za-z\s]+(?=\s\d+ g|\b)', text)
        phrases = [phrase.strip(' g\n') for phrase in phrases]
        numbers = re.findall(r'\b\d+\b', text)
        for i, phrase in enumerate(phrases):
            if phrase=='':
                break
            if i>=len(numbers):
                break
            res +=int(numbers[i]) * float(hash_table.get(phrase,2.0))#phrases_stripped
        return res#', '.join(phrases_stripped[:-1]) + phrases_stripped[-1]
    else:
        return np.nan
def extract_and_sum_numbers(s):
    try:
        numbers = re.findall(r'(\d+)', s)
        res =sum(int(num) for num in numbers)
        return float(res)
    except TypeError:
        return 0.0
def fine_preprocess(data_type,date_no,n_lag=1,type='I'):
    # 数据预处理：加载数据集
    path = f'data_set/{data_type}/processed_{date_no}.csv'
    dataset = read_csv(path, header=0, index_col=0)
    # 将 'data not available' 替换为 NaN
    dataset.loc[dataset['Dietary intake'] == 'data not available', 'Dietary intake'] = '258.44'
    dataset.loc[dataset['CSII - basal insulin (Novolin R, IU / H)']=='temporarily suspend insulin delivery','CSII - basal insulin (Novolin R, IU / H)']=0.0
    dataset.loc[dataset['CSII - bolus insulin (Novolin R, IU)']=='temporarily suspend insulin delivery','CSII - bolus insulin (Novolin R, IU)']=0.0
    
    values = dataset.values
    s_c_col = dataset.iloc[:, 5].astype(str)
    n_ha_col = dataset.iloc[:, 6].apply(extract_and_sum_numbers).astype(str)
    i_v_col = dataset.iloc[:, 9].astype(str)

    # Extract the IU values using regular expressions
    #Non-insulin hypoglycemic agents
    s_c_iu = s_c_col.str.extract(r'(\d+)\s*IU', expand=False)
    i_v_iu = i_v_col.str.extract(r'(\d+)\s*IU', expand=False)
    # Convert extracted values to numeric
    s_c_iu = pd.to_numeric(s_c_iu, errors='coerce')
    i_v_iu = pd.to_numeric(i_v_iu, errors='coerce')
    n_ha_iu = pd.to_numeric(n_ha_col, errors='coerce')
    # Add the extracted values back to the DataFrame
    dataset['IU- s.c.'] = s_c_iu
    dataset['IU- i.v.'] = i_v_iu #Really Omit!
    #dataset['Non-insulin hypoglycemic agents - mg']=n_ha_iu #Really omit!
    dataset.drop('Insulin dose - s.c.',axis=1,inplace=True)
    dataset.drop('Insulin dose - i.v.',axis=1,inplace=True)
    dataset.drop('Non-insulin hypoglycemic agents',axis=1,inplace=True)
    #dataset.drop('CBG (mg / dl)',axis=1,inplace=True)
    dataset.drop('IU- s.c.',axis=1,inplace=True)
    dataset.drop('IU- i.v.',axis=1,inplace=True)
    try:
        dataset.drop('饮食',axis=1,inplace=True) 
    except KeyError: 
        dataset.drop('进食量',axis=1,inplace=True)
    dataset.drop('Blood Ketone (mmol / L)',axis=1,inplace=True)#Really omit
    dataset['Dietary calories'] = dataset['Dietary intake'].apply(extract_dietary_words)
    #print(dataset.describe())
    dataset.drop('Dietary intake',axis=1,inplace=True)
    # Assuming dataset is your DataFrame
    dataset['CSII - basal insulin (Novolin R, IU / H)'] = dataset['CSII - basal insulin (Novolin R, IU / H)'].ffill(limit=12).astype(float)
    dataset.fillna(0, inplace=True)
    #dataset.drop('CSII - basal insulin (Novolin R, IU / H)',axis=1,inplace=True)
    #print(dataset)
    my_rfe(dataset)
    csv_file_path = 'output.csv'
    dataset.to_csv(csv_file_path, index=False)
    values = dataset.values
    #保证为float ensure all data is float
    #values = values.astype('float32')
    #归一化 normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    reframed = series_to_supervised(scaled, 1, 1,n_lag)
    reframed.drop(reframed.columns[[6,7,8,9]], axis=1, inplace=True)
    #reduce_dimension(reframed)
    #print(reframed)
    return reframed,scaler

The best split train-test retio is 4:1,I guess. 

In [4]:
def split_data(reframed):
    #数据准备
    #把数据分为训练数据和测试数据 split into train and test sets
    values = reframed.values
    #拿一年的时间长度训练
    #print(reframed.shape[0])
    n_train_quarters = int(reframed.shape[0]*0.9)
    #划分训练数据和测试数据
    train = values[:n_train_quarters, :]
    test = values[n_train_quarters:, :]
    #拆分输入输出 split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    #reshape输入为LSTM的输入格式 reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    #print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
    #print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X,train_y,test_X,test_y

# Model

This is because tensorflow can't be directly imported in python.And keras's BACKEND env should be assigned as one of the following :```tensorflow```,```fax```.

In [ ]:
!pip install tensorflow

## Training
### NOTE:
1. the batch size must be a power of 2.

Best hyperparameter ever:
- optimizer:adam,(nadam isn't that bad,but others suffer)
- epochs:20
- batch_size:16
- layers of LSTM:100
- layers of Dense must be 1

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.layers import LSTM
from matplotlib import pyplot
from keras.optimizers import Adam
##模型定义 design network
def train(data_type,date_no,train_X,train_y,test_X,test_y,n_lag=1):
    model = Sequential()
    model.add(Input(shape=(train_X.shape[1], train_X.shape[2])))
    model.add(LSTM(100))
    model.add(Dense(1))
    learning_rate = 0.001
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mae', optimizer='adam')
    #模型训练 fit network
    history = model.fit(train_X, train_y, epochs=20, batch_size=16, validation_data=(test_X, test_y), verbose=0,
                    shuffle=False)
    #输出 plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.savefig(f'data_set/trained_{data_type}_{date_no}_{n_lag}.png')
    pyplot.show()
    pyplot.close()
    return model

## Prediction

In [7]:
from numpy import concatenate
def predict(data_type,date_no,model,scaler,train_X,train_y,test_X,test_y,n_lag=1):
    #进行预测 make a prediction
    #print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    #预测数据逆缩放 invert scaling for forecast
    inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
    #print('inv_yhat')
    #print(inv_yhat.shape)
    #print(scaler.n_features_in_)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]
    inv_yhat = np.array(inv_yhat)
    #真实数据逆缩放 invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, 0]

    #画出真实数据和预测数据
    pyplot.plot(inv_yhat,label='prediction')
    pyplot.plot(inv_y,label='true')
    pyplot.legend()
    pyplot.savefig(f'predict_samples')
    pyplot.savefig(f'data_set/{data_type}/figures/{date_no}_{n_lag}.png')
    pyplot.show()
    pyplot.close()
    return inv_y, inv_yhat

# Calculate RMSE

In [8]:
from math import sqrt
# calculate RMSE
def cal_rmse(inv_y, inv_yhat):
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Test RMSE: %.3f' % rmse)
    return rmse

## Cross Validation


In [9]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
def roc_auc(y,yhat):
    # 计算AUC-ROC
    roc_auc = roc_auc_score(y, yhat)
    fpr, tpr, _ = roc_curve(y, yhat)
    roc_auc_value = auc(fpr, tpr)

    print(f'AUC-ROC: {roc_auc}')

    # 绘制ROC曲线
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_value:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
    plt.close()

## Enumerate all files

In [ ]:
import os
total = 0
cnt = 0
def lstm_file(data_type,date_no,n_lag=1,type='I'):
    global total, cnt
    df=get_dataframe(data_type,date_no)
    rf,scaler=fine_preprocess(data_type,date_no,n_lag,type)
    train_X,train_y,test_X,test_y=split_data(rf)
    model=train(data_type,date_no,train_X,train_y,test_X,test_y,n_lag)
    inv_y, inv_yhat=predict(data_type,date_no,model,scaler,train_X,train_y,test_X,test_y,n_lag)
    res=cal_rmse(inv_y, inv_yhat)
    #print(res)
    total = total+res
    cnt = cnt+1
    return res
    #roc_auc(inv_y, inv_yhat)
file_path = "result.txt"
with open(file_path, "w", encoding="utf-8") as file:
    pass
for i in range(4):
    number_to_append = 42
    for root, dirs, files in os.walk('Shanghai_T1DM'):
            for file in files:
                res =lstm_file('Shanghai_T1DM',file,i,'I')
                with open(file_path, "a", encoding="utf-8") as file:
                    file.write(f"I,{i} step,{res}\n")

    with open(file_path, "a", encoding="utf-8") as file:
        file.write(f"Summary:I,{i} step,{total/cnt}\n")
    total = 0
    cnt = 0
    for root, dirs, files in os.walk('Shanghai_T2DM'):
            for file in files:
                res =lstm_file('Shanghai_T2DM',file,i,'II')
                with open(file_path, "a", encoding="utf-8") as file:
                    file.write(f"II,{i} step,{res}\n")
    with open(file_path, "a", encoding="utf-8") as file:
        file.write(f"Summary:II,{i} step,{total/cnt}\n")